In [8]:
import requests
import pandas as pd
import time

COMPOUND_V2_SUBGRAPH_URL = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"
COMPOUND_V3_SUBGRAPH_URL = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v3"

FEATURE_WEIGHTS = {
    'num_liquidations': 0.30,
    'late_repayment_ratio': 0.20,
    'avg_collateral_utilization': 0.15,
    'borrow_to_repay_ratio': 0.15,
    'largest_borrow': 0.10,
    'activity_duration_days': 0.10
}

def run_query(url, query):
    """Run a GraphQL query with retry."""
    for _ in range(3):
        try:
            r = requests.post(url, json={'query': query})
            if r.status_code == 200:
                return r.json()
        except Exception as e:
            print(f"Query error: {e}")
        time.sleep(1)
    raise Exception(f"GraphQL query failed after retries for URL {url}")

def fetch_wallet_data(wallet):
    """Fetch Compound V2 and V3 data/events for a wallet."""
    query_template = '''
    {{
      account(id: "{wallet}") {{
        borrowEvents {{ amount transaction {{ timestamp }} }}
        repayEvents {{ amount transaction {{ timestamp }} }}
        liquidateEvents {{ amount transaction {{ timestamp }} }}
        transactions {{ transaction {{ timestamp }} }}
      }}
    }}
    '''
    wallet_lower = wallet.lower()
    v2_data = run_query(COMPOUND_V2_SUBGRAPH_URL, query_template.format(wallet=wallet_lower))
    v3_data = run_query(COMPOUND_V3_SUBGRAPH_URL, query_template.format(wallet=wallet_lower))

    account_v2 = v2_data.get('data', {}).get('account') or {}
    account_v3 = v3_data.get('data', {}).get('account') or {}

    return account_v2, account_v3

def process_wallet_features(wallet):
    """Encode wallet risk features based on Compound protocol activity."""
    v2, v3 = fetch_wallet_data(wallet)

    borrow_events = (v2.get('borrowEvents') or []) + (v3.get('borrowEvents') or [])
    repay_events = (v2.get('repayEvents') or []) + (v3.get('repayEvents') or [])
    liquidate_events = (v2.get('liquidateEvents') or []) + (v3.get('liquidateEvents') or [])
    transactions = (v2.get('transactions') or []) + (v3.get('transactions') or [])

    num_liquidations = len(liquidate_events)
    num_borrows = len(borrow_events)
    num_repays = len(repay_events)


    largest_borrow = max([float(ev['amount']) for ev in borrow_events], default=0)

    timestamps = [int(tx['transaction']['timestamp']) for tx in transactions if tx.get('transaction')]
    activity_duration_days = ((max(timestamps) - min(timestamps)) / 86400) if timestamps else 0

    total_borrowed = sum(float(ev['amount']) for ev in borrow_events)
    total_repaid = sum(float(ev['amount']) for ev in repay_events)
    borrow_to_repay_ratio = (total_borrowed / total_repaid) if total_repaid > 0 else total_borrowed

    late_repayment_ratio = ((num_borrows - num_repays) / num_borrows) if num_borrows > 0 else 0

    avg_collateral_utilization = min(borrow_to_repay_ratio, 1.0)


    return {
        'wallet_id': wallet,
        'num_liquidations': num_liquidations,
        'late_repayment_ratio': late_repayment_ratio,
        'avg_collateral_utilization': avg_collateral_utilization,
        'borrow_to_repay_ratio': borrow_to_repay_ratio,
        'largest_borrow': largest_borrow,
        'activity_duration_days': activity_duration_days
    }

def normalize_features(df, invert_features=[]):
    """Normalize features between 0 and 1, optionally invert some."""
    df_norm = pd.DataFrame()
    for col in df.columns:
        min_val = df[col].min()
        max_val = df[col].max()
        if min_val == max_val:
            df_norm[col] = 0.0
        else:
            df_norm[col] = (df[col] - min_val) / (max_val - min_val)
        if col in invert_features:
            df_norm[col] = 1 - df_norm[col]
    return df_norm

def calculate_risk_score(row):
    """Compute weighted risk score scaled to 0-1000."""
    score = 0.0
    for feature, weight in FEATURE_WEIGHTS.items():

        if feature in row:
            score += row[feature] * weight
        else:
            print(f"Warning: Feature '{feature}' not found in row.")
    return int(score * 1000)

def main():
    wallets_df = pd.read_csv("/content/Wallet id - Sheet1.csv")
    wallets = wallets_df['wallet_id'].tolist()

    records = []
    for idx, wallet in enumerate(wallets, start=1):
        print(f"Processing wallet {idx}/{len(wallets)}: {wallet}")
        try:
            features = process_wallet_features(wallet)
            records.append(features)
        except Exception as e:
            print(f"Error processing {wallet}: {e}")

    df_features = pd.DataFrame(records).set_index('wallet_id')

    invert_feats = ['activity_duration_days']

    df_norm = normalize_features(df_features, invert_features=invert_feats)

    df_norm['score'] = [calculate_risk_score(row) for index, row in df_norm.iterrows()]

    result = df_norm[['score']].reset_index()
    result.to_csv("wallet_risk_scores.csv", index=False)
    print("Saved wallet risk scores: wallet_risk_scores.csv")

if __name__ == "__main__":
    main()

Processing wallet 1/103: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Processing wallet 2/103: 0x06b51c6882b27cb05e712185531c1f74996dd988
Processing wallet 3/103: 0x0795732aacc448030ef374374eaae57d2965c16c
Processing wallet 4/103: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Processing wallet 5/103: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Processing wallet 6/103: 0x104ae61d8d487ad689969a17807ddc338b445416
Processing wallet 7/103: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
Processing wallet 8/103: 0x124853fecb522c57d9bd5c21231058696ca6d596
Processing wallet 9/103: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
Processing wallet 10/103: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
Processing wallet 11/103: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
Processing wallet 12/103: 0x19df3e87f73c4aaf4809295561465b993e102668
Processing wallet 13/103: 0x1ab2ccad4fc97c9968ea87d4435326715be32872
Processing wallet 14/103: 0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f
Processing wallet 15/103: 0x1e43dacdcf86367